In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi

Sun May 16 09:43:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Install these packages with these specific versions else the notebook breaks
!pip install transformers==4.5.1
!pip install pytorch_lightning==1.2.10
!pip install sentencepiece

     |████████████████████████████████| 2.1MB 7.6MB/s 
     |████████████████████████████████| 3.3MB 64.7MB/s 
     |████████████████████████████████| 901kB 61.6MB/s 
     |████████████████████████████████| 849kB 7.5MB/s 
     |████████████████████████████████| 276kB 20.9MB/s 
     |████████████████████████████████| 184kB 22.1MB/s 
     |████████████████████████████████| 112kB 24.5MB/s 
     |████████████████████████████████| 829kB 18.2MB/s 
     |████████████████████████████████| 1.3MB 37.6MB/s 
     |████████████████████████████████| 296kB 43.2MB/s 
     |████████████████████████████████| 143kB 51.2MB/s 
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=af4223dce3bf65f4b0c5016e5dc2f938ceba453aafe90ba59173f6acde9cb194
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=041ce475313ed54f26a274ebc02d1e

In [4]:
# Import packages
import argparse
import glob
import pickle
import os
import json
import time
import logging
import random
import re
from tqdm import tqdm
from itertools import chain
from string import punctuation
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import sentencepiece


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
import csv
from dataclasses import dataclass

from enum import Enum
from typing import List, Optional
from transformers import PreTrainedTokenizer

In [6]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [7]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [8]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    
    if type(hparams) is dict: 
      hparams = argparse.Namespace(**hparams)
    
    self.hparams = hparams
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    
    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('training_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    self.log('avg_training_loss', avg_train_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"val_loss": loss}
    return {"val_loss": loss, "log": tensorboard_logs}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    print(avg_loss)
    tensorboard_logs = {"val_loss": avg_loss}
    self.log('avg_val_loss', avg_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self,
                     epoch=None, 
                     batch_idx=None, 
                     optimizer=None, 
                     optimizer_idx=None, 
                     optimizer_closure=None, 
                     on_tpu=None, 
                     using_native_amp=None, 
                     using_lbfgs=None
                     ):

    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [9]:
# The below code is adapted from:
# https://github.com/huggingface/transformers/blob/master/examples/multiple-choice/utils_multiple_choice.py

@dataclass(frozen=True)
class InputExample:
    """
    A single training/test example for multiple choice
    Args:
        example_id: Unique id for the example.
        question: string. The untokenized text of the second sequence (question).
        contexts: list of str. The untokenized text of the first sequence (context of corresponding question).
        endings: list of str. multiple choice's options. Its length must be equal to contexts' length.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """

    name: str
    ingredients: str
    steps: List[str]
    label: Optional[str]

class Split(Enum):
    train = "train"
    dev = "dev"
    test = "test"

class DataProcessor:
    """Base class for data converters for multiple choice data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

class RecipeProcessor(DataProcessor):
    """Processor for the SWAG data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} train".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "train.csv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "val.csv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        raise ValueError(
            "For swag testing, the input file does not contain a label column. It can not be tested in current code"
            "setting!"
        )
        return self._create_examples(self._read_csv(os.path.join(data_dir, "test.csv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2", "3"]

    def _read_csv(self, input_file):
        with open(input_file, "r", encoding="utf-8") as f:
            return list(csv.reader(f))

    def _create_examples(self, lines: List[List[str]], type: str):
        """Creates examples for the training and dev sets."""

        examples = [
            InputExample(
                name=line[1],
                # common beginning of each
                # choice is stored in "sent2".
                ingredients=eval(line[3]),
                steps=eval(line[2]),
                label=eval(line[2]),
            )
            for line in lines[1:]  # we skip the line with the column names
        ]

        return examples

In [10]:
class RecipeDataset(Dataset):
    
    def __init__(self, tokenizer, data_dir, type_path,  max_len=512, mask_percent=0.5):
        self.data_dir = data_dir
        self.type_path = type_path
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.ingredients = []
        self.mask_percent = mask_percent
        
        self.proc = RecipeProcessor()
        self._build()
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def __len__(self):
        return len(self.inputs)
  
    def _build(self):
        if self.type_path == 'train':
            examples = self.proc.get_train_examples(self.data_dir)
        else:
            examples = self.proc.get_dev_examples(self.data_dir)
    
        for i, example in enumerate(examples):
            if i % 10000 == 0:
                print(i)
            self._create_features(example)
    
    
    def _create_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          mask_words = np.round(self.mask_percent*total_words).astype(int)
          input_words = words[:mask_words] + ['<extra_id_0>']
          label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_features(self, example):
    
        ingredients_ = ",".join(example.ingredients)
        name_ = example.name
        masked_, label_ = self._create_completion(example.steps)
                                        
        input_ = "name: %s  ingredients: %s masked: %s </s>" % (name_, ingredients_, masked_)
        target =  label_ + " </s>"
        
        # tokenize inputs
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input_], max_length = 128, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )
        # tokenize targets
        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target], max_length = 75, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)

In [11]:
def get_dataset(tokenizer, type_path, args):
  return RecipeDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)

In [12]:
args_dict = dict(
    data_dir='drive/MyDrive/recipe-generation/data', # path for data files
    output_dir='drive/MyDrive/recipe-generation/t5_fill_blanks', # path to save the checkpoints
    model_name_or_path='t5-small',
    tokenizer_name_or_path='t5-base',
    max_seq_length=512,
    learning_rate=3e-4,
    weight_decay=0.05,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=8,
    eval_batch_size=8,
    num_train_epochs=7,
    gradient_accumulation_steps=1,
    n_gpu=1,
    early_stop_callback=True,
    fp_16=False, # if you want to enable 16-bit training then install apex and set this to true
    opt_level='O1', # you can find out more on optimisation levels here https://nvidia.github.io/apex/amp.html#opt-levels-and-properties
    max_grad_norm=1.0, # if you enable 16-bit training then set this to a sensible value, 0.5 is a good default
    seed=42,
    cp_filename='latest'
)


args = argparse.Namespace(**args_dict)
print(args_dict)

{'data_dir': 'drive/MyDrive/recipe-generation/data', 'output_dir': 'drive/MyDrive/recipe-generation/t5_fill_blanks', 'model_name_or_path': 't5-small', 'tokenizer_name_or_path': 't5-base', 'max_seq_length': 512, 'learning_rate': 0.0003, 'weight_decay': 0.05, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 8, 'eval_batch_size': 8, 'num_train_epochs': 7, 'gradient_accumulation_steps': 1, 'n_gpu': 1, 'early_stop_callback': True, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42, 'cp_filename': 'latest'}


In [13]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath=args.output_dir, monitor="avg_val_loss", mode="min", save_top_k=1, 
    filename=args.cp_filename
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
    val_check_interval=5000,
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Checkpoint directory drive/MyDrive/recipe-generation/t5_fill_blanks exists and is not empty.
  warnings.warn(*args, **kwargs)


In [14]:
# Training loop
# Set these appropriately depending on whether you want to execute training and freezing
train = True
freeze = True
# Start training
if train:  
  # Load model
  model = T5FineTuner(args)
  # Freeze encoder
  if freeze:
    for param in model.model.encoder.parameters():
      param.requires_grad = False

trainer = pl.Trainer(**train_params)
trainer.fit(model)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 60.5 M
-----------------------------------------------------
25.2 M    Trainable params
35.3 M    Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)


0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


10000


...

tensor(1.9834, device='cuda:0')


tensor(1.9773, device='cuda:0')


tensor(1.9772, device='cuda:0')


tensor(1.9724, device='cuda:0')



1